In [1]:
## ARM Code


In [ ]:
# Webots + IK Controller
from controller import Supervisor
import numpy as np
from scipy.optimize import minimize

# --- Webots Setup ---
robot = Supervisor()
TIME_STEP = 64

arm_joint_names = ['arm1', 'arm2', 'arm3', 'arm4', 'arm5']
gripper_joint_names = ['finger::left', 'finger::right']

arm_motors = [robot.getDevice(name) for name in arm_joint_names]
gripper_motors = [robot.getDevice(name) for name in gripper_joint_names]

ARM_SPEED = 0.8
GRIPPER_SPEED = 0.2

for motor in arm_motors:
    motor.setVelocity(ARM_SPEED)
for motor in gripper_motors:
    motor.setVelocity(GRIPPER_SPEED)

youbot_node = robot.getFromDef("youBot")
translation_field = youbot_node.getField("translation")

# --- Link Dimensions ---
link1_size = 0.0756479255
link2_size = 0.155000
link3_size = 0.135000
link4_size = 0.174510

# --- Transform Utilities ---
def translate(x, y, z):
    T = np.eye(4)
    T[:3, 3] = [x, y, z]
    return T

def rotate_z(theta):
    R = np.eye(4)
    c, s = np.cos(theta), np.sin(theta)
    R[0:2, 0:2] = [[c, -s], [s, c]]
    return R

def rotate_y(theta):
    R = np.eye(4)
    c, s = np.cos(theta), np.sin(theta)
    R[0, 0], R[0, 2], R[2, 0], R[2, 2] = c, s, -s, c
    return R

# --- Forward Kinematics ---
def forward_kinematics(theta):
    theta1, theta2, theta3, theta4 = theta
    position = translation_field.getSFVec3f()
    T_base = translate(position[0], position[1], position[2])
    T1 = T_base @ rotate_z(theta1)
    T2 = T1 @ translate(link1_size, 0, 0) @ rotate_y(theta2)
    T3 = T2 @ translate(link2_size, 0, 0) @ rotate_y(theta3)
    T4 = T3 @ translate(link3_size, 0, 0) @ rotate_y(theta4)
    end_effector = T4 @ np.array([link4_size, 0, 0, 1])
    return end_effector[:3]

# --- IK Objective ---
def ik_objective(theta, target):
    eff_pos = forward_kinematics(theta)
    return np.linalg.norm(eff_pos - target)

# --- Inverse Kinematics Solver ---
def solve_ik(target, initial_guess):
    bounds = [
        (-2.9496, 2.949600031075391),                 # theta1
        (-1.1344612254017445, 1.5708),              # theta2
        (-2.63545, 2.54818),                 # theta3
        (-1.78024, 1.78024)       # theta4
    ]
    result = minimize(
        ik_objective,
        initial_guess,
        args=(target,),
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 1000, 'disp': False}
    )
    theta = result.x
    final_pos = forward_kinematics(theta)
    if np.linalg.norm(final_pos - target) < 1e-2:
        return theta
    return None

# --- Reachability Check ---
def is_target_reachable(target, center, max_radius=0.53832895951):
    return np.linalg.norm(target - center) <= max_radius

# --- Arm Base Position ---
def get_arm_base_position():
    position = translation_field.getSFVec3f()
    return np.array(position) + np.array([0.156, 0, 0])

# --- Gripper Control ---
def set_gripper_position(position):
    for motor in gripper_motors:
        motor.setPosition(position)

# --- Move Arm to Target ---
def move_arm_to_theta(theta):
    # Ensure theta has 5 values (last joint dummy or fixed)
    theta_full = list(theta) + [0.0]
    for i, motor in enumerate(arm_motors):
        motor.setPosition(theta_full[i])
    # Wait for arm to move
    for _ in range(int(3000 / TIME_STEP)):
        robot.step(TIME_STEP)

# --- Main Execution ---

# 1. Move arm to initial target position
initial_target = [0.0, 1.1, 0.5, 1.1, 0.0]
print("Moving arm to initial target position...")
for i, motor in enumerate(arm_motors):
    motor.setPosition(initial_target[i])

# Wait for arm to reach initial position
for _ in range(int(3000 / TIME_STEP)):
    robot.step(TIME_STEP)


initial_theta = [0.0, 1.1, 0.5, 1.1]
target = np.array([0.172056, 0.303037, 0.3030372])
position = translation_field.getSFVec3f()
print(position)
if not is_target_reachable(target, position):
    print("❌ Target is unreachable.")
else:
    final_theta = solve_ik(target, initial_theta)
    if final_theta is not None:
        print("✅ Final joint angles:", final_theta)
        print("Moving robot to final configuration...")
        move_arm_to_theta(final_theta)

        print("Opening gripper...")
        set_gripper_position(0.02)
        for _ in range(30):
            robot.step(TIME_STEP)

        print("Closing gripper...")
        set_gripper_position(0.0)
        for _ in range(30):
            robot.step(TIME_STEP)

        print("✅ Done.")
    else:
        print("❌ IK failed to find solution.")

# Hold simulation
while robot.step(TIME_STEP) != -1:
    pass


## base code

In [ ]:
from controller import Supervisor
import math
import heapq
import numpy as np

# Constants
TIME_STEP = 32
MAP_SIZE = 100
MAP_RES = 0.05  # 5cm per cell
LIDAR_RANGE = 1.5
ROBOT_RADIUS = 0.2
MAX_SPEED = 10.28
TARGET = [1.16, 0]
STOP_OFFSET = 0.5  # Stop 30 cm before target

# Supervisor init
robot = Supervisor()
lidar = robot.getDevice("lidar")
lidar.enable(TIME_STEP)
lidar.enablePointCloud()

robot_node = robot.getFromDef("YOUBOT")
translation_field = robot_node.getField("translation")
rotation_field = robot_node.getField("rotation")

wheels = [robot.getDevice(name) for name in ["wheel1", "wheel2", "wheel3", "wheel4"]]
for motor in wheels:
    motor.setPosition(float('inf'))
    motor.setVelocity(0.0)

# Map grid: 0 = free, 1 = obstacle
grid = np.zeros((MAP_SIZE, MAP_SIZE), dtype=int)

def world_to_grid(x, y):
    gx = int((x + MAP_SIZE * MAP_RES / 2) / MAP_RES)
    gy = int((y + MAP_SIZE * MAP_RES / 2) / MAP_RES)
    return gx, gy

def grid_to_world(gx, gy):
    x = gx * MAP_RES - MAP_SIZE * MAP_RES / 2
    y = gy * MAP_RES - MAP_SIZE * MAP_RES / 2
    return x, y

def update_occupancy_grid(pos, lidar_vals):
    robot_x, robot_y = pos
    angle_step = lidar.getFov() / lidar.getHorizontalResolution()
    for i, dist in enumerate(lidar_vals):
        if dist > LIDAR_RANGE:
            continue
        angle = -lidar.getFov() / 2 + i * angle_step
        lx = robot_x + dist * math.cos(robot_theta + angle)
        ly = robot_y + dist * math.sin(robot_theta + angle)
        gx, gy = world_to_grid(lx, ly)
        if 0 <= gx < MAP_SIZE and 0 <= gy < MAP_SIZE:
            grid[gx, gy] = 1

def a_star(start, goal):
    sx, sy = world_to_grid(*start)
    gx, gy = world_to_grid(*goal)

    def heuristic(a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    open_set = []
    heapq.heappush(open_set, (0, (sx, sy)))
    came_from = {}
    g_score = { (sx, sy): 0 }

    while open_set:
        _, current = heapq.heappop(open_set)
        if current == (gx, gy):
            break
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            neighbor = (current[0]+dx, current[1]+dy)
            if 0 <= neighbor[0] < MAP_SIZE and 0 <= neighbor[1] < MAP_SIZE:
                if grid[neighbor] == 1:  # Obstacle
                    continue
                tentative = g_score[current] + 1
                if tentative < g_score.get(neighbor, float('inf')):
                    g_score[neighbor] = tentative
                    priority = tentative + heuristic(neighbor, (gx, gy))
                    heapq.heappush(open_set, (priority, neighbor))
                    came_from[neighbor] = current

    path = []
    node = (gx, gy)
    while node in came_from:
        path.append(grid_to_world(*node))
        node = came_from[node]
    path.reverse()
    return path

def compute_wheel_velocities(vx, vy, omega):
    l = 0.235
    w = 0.15
    r = 0.05
    return [
        (vx + vy + (l + w) * omega) / r,
        (vx - vy - (l + w) * omega) / r,
        (vx - vy + (l + w) * omega) / r,
        (vx + vy - (l + w) * omega) / r
    ]

# Helper function to compute adjusted target based on stop offset
def compute_adjusted_target(robot_pos, target, offset):
    dx = target[0] - robot_pos[0]
    dy = target[1] - robot_pos[1]
    distance = math.hypot(dx, dy)
    if distance <= offset:
        # Already closer than offset, return original target to avoid division by zero
        return target
    ratio = (distance - offset) / distance
    adjusted_x = robot_pos[0] + dx * ratio
    adjusted_y = robot_pos[1] + dy * ratio
    return [adjusted_x, adjusted_y]

# Main loop
path = []
path_index = 0
while robot.step(TIME_STEP) != -1:
    pos = translation_field.getSFVec3f()
    rot = rotation_field.getSFRotation()
    robot_theta = rot[3] if rot[2] >= 0 else -rot[3]
    robot_pos = (pos[0], pos[1])

    # Compute adjusted target point with offset
    adjusted_target = compute_adjusted_target(robot_pos, TARGET, STOP_OFFSET)

    lidar_vals = lidar.getRangeImage()
    update_occupancy_grid(robot_pos, lidar_vals)

    # Plan only once or when path exhausted
    if not path or path_index >= len(path):
        path = a_star(robot_pos, adjusted_target)
        path_index = 0
        if not path:
            print("❌ No path found!")
            break

    # Follow path
    target_wp = path[path_index]
    dx = target_wp[0] - robot_pos[0]
    dy = target_wp[1] - robot_pos[1]
    dist = math.hypot(dx, dy)

    if dist < 0.05 and path_index < len(path) - 1:
        path_index += 1
        continue

    # PID-like controller to move toward next waypoint
    angle_to_wp = math.atan2(dy, dx)
    angle_diff = math.atan2(math.sin(angle_to_wp - robot_theta), math.cos(angle_to_wp - robot_theta))
    vx = 0.5 * math.cos(angle_diff)
    vy = 0.5 * math.sin(angle_diff)
    omega = 2.0 * angle_diff

    speeds = compute_wheel_velocities(vx, vy, omega)
    for i in range(4):
        wheels[i].setVelocity(max(min(speeds[i], MAX_SPEED), -MAX_SPEED))

    # Stop when close enough to adjusted target
    distance_to_adjusted_target = math.hypot(robot_pos[0] - adjusted_target[0], robot_pos[1] - adjusted_target[1])
    if distance_to_adjusted_target < 0.05:
        for motor in wheels:
            motor.setVelocity(0.0)
        print("✅ Stopped at offset before target.")
        break

ModuleNotFoundError: No module named 'controller'

## Merged code

In [ ]:
from controller import Supervisor
import math
import heapq
import numpy as np
from scipy.optimize import minimize


# Constants
TIME_STEP = 32
MAP_SIZE = 100
MAP_RES = 0.05  # 5cm per cell
LIDAR_RANGE = 1.5
ROBOT_RADIUS = 0.2
MAX_SPEED = 10.28
TARGET = [1.16, 0]
STOP_OFFSET = 0.5  # Stop 30 cm before target
TIME_STEP = 64
ARM_SPEED = 0.8
GRIPPER_SPEED = 0.2


# Supervisor init
robot = Supervisor()
lidar = robot.getDevice("lidar")
lidar.enable(TIME_STEP)
lidar.enablePointCloud()


arm_joint_names = ['arm1', 'arm2', 'arm3', 'arm4', 'arm5']
gripper_joint_names = ['finger::left', 'finger::right']
robot_node = robot.getFromDef("YOUBOT")
translation_field = robot_node.getField("translation")
rotation_field = robot_node.getField("rotation")

arm_motors = [robot.getDevice(name) for name in arm_joint_names]
gripper_motors = [robot.getDevice(name) for name in gripper_joint_names]
wheels = [robot.getDevice(name) for name in ["wheel1", "wheel2", "wheel3", "wheel4"]]

for motor in arm_motors:
    motor.setVelocity(ARM_SPEED)
for motor in gripper_motors:
    motor.setVelocity(GRIPPER_SPEED)
for motor in wheels:
    motor.setPosition(float('inf'))
    motor.setVelocity(0.0)


youbot_node = robot.getFromDef("youBot")
translation_field = youbot_node.getField("translation")

# Map grid: 0 = free, 1 = obstacle
grid = np.zeros((MAP_SIZE, MAP_SIZE), dtype=int)

def world_to_grid(x, y):
    gx = int((x + MAP_SIZE * MAP_RES / 2) / MAP_RES)
    gy = int((y + MAP_SIZE * MAP_RES / 2) / MAP_RES)
    return gx, gy

def grid_to_world(gx, gy):
    x = gx * MAP_RES - MAP_SIZE * MAP_RES / 2
    y = gy * MAP_RES - MAP_SIZE * MAP_RES / 2
    return x, y

def update_occupancy_grid(pos, lidar_vals):
    robot_x, robot_y = pos
    angle_step = lidar.getFov() / lidar.getHorizontalResolution()
    for i, dist in enumerate(lidar_vals):
        if dist > LIDAR_RANGE:
            continue
        angle = -lidar.getFov() / 2 + i * angle_step
        lx = robot_x + dist * math.cos(robot_theta + angle)
        ly = robot_y + dist * math.sin(robot_theta + angle)
        gx, gy = world_to_grid(lx, ly)
        if 0 <= gx < MAP_SIZE and 0 <= gy < MAP_SIZE:
            grid[gx, gy] = 1

def a_star(start, goal):
    sx, sy = world_to_grid(*start)
    gx, gy = world_to_grid(*goal)

    def heuristic(a, b):
        return abs(a[0]-b[0]) + abs(a[1]-b[1])

    open_set = []
    heapq.heappush(open_set, (0, (sx, sy)))
    came_from = {}
    g_score = { (sx, sy): 0 }

    while open_set:
        _, current = heapq.heappop(open_set)
        if current == (gx, gy):
            break
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            neighbor = (current[0]+dx, current[1]+dy)
            if 0 <= neighbor[0] < MAP_SIZE and 0 <= neighbor[1] < MAP_SIZE:
                if grid[neighbor] == 1:  # Obstacle
                    continue
                tentative = g_score[current] + 1
                if tentative < g_score.get(neighbor, float('inf')):
                    g_score[neighbor] = tentative
                    priority = tentative + heuristic(neighbor, (gx, gy))
                    heapq.heappush(open_set, (priority, neighbor))
                    came_from[neighbor] = current

    path = []
    node = (gx, gy)
    while node in came_from:
        path.append(grid_to_world(*node))
        node = came_from[node]
    path.reverse()
    return path

def compute_wheel_velocities(vx, vy, omega):
    l = 0.235
    w = 0.15
    r = 0.05
    return [
        (vx + vy + (l + w) * omega) / r,
        (vx - vy - (l + w) * omega) / r,
        (vx - vy + (l + w) * omega) / r,
        (vx + vy - (l + w) * omega) / r
    ]

# Helper function to compute adjusted target based on stop offset
def compute_adjusted_target(robot_pos, target, offset):
    dx = target[0] - robot_pos[0]
    dy = target[1] - robot_pos[1]
    distance = math.hypot(dx, dy)
    if distance <= offset:
        # Already closer than offset, return original target to avoid division by zero
        return target
    ratio = (distance - offset) / distance
    adjusted_x = robot_pos[0] + dx * ratio
    adjusted_y = robot_pos[1] + dy * ratio
    return [adjusted_x, adjusted_y]

# --- Link Dimensions ---
link1_size = 0.0756479255
link2_size = 0.155000
link3_size = 0.135000
link4_size = 0.174510

# --- Transform Utilities ---
def translate(x, y, z):
    T = np.eye(4)
    T[:3, 3] = [x, y, z]
    return T

def rotate_z(theta):
    R = np.eye(4)
    c, s = np.cos(theta), np.sin(theta)
    R[0:2, 0:2] = [[c, -s], [s, c]]
    return R

def rotate_y(theta):
    R = np.eye(4)
    c, s = np.cos(theta), np.sin(theta)
    R[0, 0], R[0, 2], R[2, 0], R[2, 2] = c, s, -s, c
    return R

# --- Forward Kinematics ---
def forward_kinematics(theta):
    theta1, theta2, theta3, theta4 = theta
    position = translation_field.getSFVec3f()
    T_base = translate(position[0], position[1], position[2])
    T1 = T_base @ rotate_z(theta1)
    T2 = T1 @ translate(link1_size, 0, 0) @ rotate_y(theta2)
    T3 = T2 @ translate(link2_size, 0, 0) @ rotate_y(theta3)
    T4 = T3 @ translate(link3_size, 0, 0) @ rotate_y(theta4)
    end_effector = T4 @ np.array([link4_size, 0, 0, 1])
    return end_effector[:3]

# --- IK Objective ---
def ik_objective(theta, target):
    eff_pos = forward_kinematics(theta)
    return np.linalg.norm(eff_pos - target)

# --- Inverse Kinematics Solver ---
def solve_ik(target, initial_guess):
    bounds = [
        (-2.9496, 2.949600031075391),                 # theta1
        (-1.1344612254017445, 1.5708),              # theta2
        (-2.63545, 2.54818),                 # theta3
        (-1.78024, 1.78024)       # theta4
    ]
    result = minimize(
        ik_objective,
        initial_guess,
        args=(target,),
        method='L-BFGS-B',
        bounds=bounds,
        options={'maxiter': 1000, 'disp': False}
    )
    theta = result.x
    final_pos = forward_kinematics(theta)
    if np.linalg.norm(final_pos - target) < 1e-2:
        return theta
    return None

# --- Reachability Check ---
def is_target_reachable(target, center, max_radius=0.53832895951):
    return np.linalg.norm(target - center) <= max_radius

# --- Arm Base Position ---
def get_arm_base_position():
    position = translation_field.getSFVec3f()
    return np.array(position) + np.array([0.156, 0, 0])

# --- Gripper Control ---
def set_gripper_position(position):
    for motor in gripper_motors:
        motor.setPosition(position)

# --- Move Arm to Target ---
def move_arm_to_theta(theta):
    # Ensure theta has 5 values (last joint dummy or fixed)
    theta_full = list(theta) + [0.0]
    for i, motor in enumerate(arm_motors):
        motor.setPosition(theta_full[i])
    # Wait for arm to move
    for _ in range(int(3000 / TIME_STEP)):
        robot.step(TIME_STEP)


# Main loop
path = []
path_index = 0
while robot.step(TIME_STEP) != -1:
    pos = translation_field.getSFVec3f()
    rot = rotation_field.getSFRotation()
    robot_theta = rot[3] if rot[2] >= 0 else -rot[3]
    robot_pos = (pos[0], pos[1])

    # Compute adjusted target point with offset
    adjusted_target = compute_adjusted_target(robot_pos, TARGET, STOP_OFFSET)

    lidar_vals = lidar.getRangeImage()
    update_occupancy_grid(robot_pos, lidar_vals)

    # Plan only once or when path exhausted
    if not path or path_index >= len(path):
        path = a_star(robot_pos, adjusted_target)
        path_index = 0
        if not path:
            print("❌ No path found!")
            break

    # Follow path
    target_wp = path[path_index]
    dx = target_wp[0] - robot_pos[0]
    dy = target_wp[1] - robot_pos[1]
    dist = math.hypot(dx, dy)

    if dist < 0.05 and path_index < len(path) - 1:
        path_index += 1
        continue

    # PID-like controller to move toward next waypoint
    angle_to_wp = math.atan2(dy, dx)
    angle_diff = math.atan2(math.sin(angle_to_wp - robot_theta), math.cos(angle_to_wp - robot_theta))
    vx = 0.5 * math.cos(angle_diff)
    vy = 0.5 * math.sin(angle_diff)
    omega = 2.0 * angle_diff

    speeds = compute_wheel_velocities(vx, vy, omega)
    for i in range(4):
        wheels[i].setVelocity(max(min(speeds[i], MAX_SPEED), -MAX_SPEED))

    # Stop when close enough to adjusted target
    distance_to_adjusted_target = math.hypot(robot_pos[0] - adjusted_target[0], robot_pos[1] - adjusted_target[1])
    if distance_to_adjusted_target < 0.05:
        for motor in wheels:
            motor.setVelocity(0.0)
        print("✅ Stopped at offset before target.")
# Wait for arm to reach initial position
for _ in range(int(3000 / TIME_STEP)):
    robot.step(TIME_STEP)


initial_theta = [0.0, 1.1, 0.5, 1.1]
target = np.array([0.172056, 0.303037, 0.3030372])
position = translation_field.getSFVec3f()
print(position)
if not is_target_reachable(target, position):
    print("❌ Target is unreachable.")
else:
    final_theta = solve_ik(target, initial_theta)
    if final_theta is not None:
        print("✅ Final joint angles:", final_theta)
        print("Moving robot to final configuration...")
        move_arm_to_theta(final_theta)

        print("Opening gripper...")
        set_gripper_position(0.02)
        for _ in range(30):
            robot.step(TIME_STEP)

        print("Closing gripper...")
        set_gripper_position(0.0)
        for _ in range(30):
            robot.step(TIME_STEP)

        print("✅ Done.")
    else:
        print("❌ IK failed to find solution.")


# Hold simulation
while robot.step(TIME_STEP) != -1:
    pass

        break

ModuleNotFoundError: No module named 'controller'